**Generate Train**

In [ ]:
import os
import numpy as np
import h5py
from PIL import Image
from skimage.color import rgb2ycbcr
from skimage.transform import resize

# Settings
folder = '/content/drive/MyDrive/Data/Train'
savepath = '/content/Results/train.h5'
size_input = 33
size_label = 21
scale = 3
stride = 14

# Initialization
data = []
label = []
padding = abs(size_input - size_label) // 2
count = 0

# Generate data
filepaths = [f for f in os.listdir(folder) if f.endswith('.bmp')]

# Function to crop images to be divisible by modulo
def modcrop(imgs, modulo):
    if len(imgs.shape) == 2:  # Grayscale
        sz = np.array(imgs.shape)
        sz = sz - (sz % modulo)
        imgs = imgs[:sz[0], :sz[1]]
    else:  # RGB or other
        sz = np.array(imgs.shape[:2])
        sz = sz - (sz % modulo)
        imgs = imgs[:sz[0], :sz[1], :]
    return imgs

for filepath in filepaths:
    image = Image.open(os.path.join(folder, filepath))
    image = np.array(image.convert('RGB'))
    image = rgb2ycbcr(image)[:, :, 0] / 255.0  # Normalize to [0, 1]

    # Crop image to be divisible by scale
    im_label = modcrop(image, scale)
    hei, wid = im_label.shape
    im_input = resize(im_label, (hei // scale, wid // scale), mode='reflect', anti_aliasing=True)
    im_input = resize(im_input, (hei, wid), mode='reflect', anti_aliasing=True)

    for x in range(0, hei - size_input + 1, stride):
        for y in range(0, wid - size_input + 1, stride):
            subim_input = im_input[x:x+size_input, y:y+size_input]
            subim_label = im_label[x+padding:x+padding+size_label, y+padding:y+padding+size_label]

            data.append(subim_input)
            label.append(subim_label)
            count += 1

# Shuffle data
order = np.random.permutation(count)
data = np.array(data)[order]
label = np.array(label)[order]

# Writing to HDF5
chunksz = 128
totalct = 0
created_flag = False

with h5py.File(savepath, 'w') as f:
    # Initialize dataset with shape based on count
    data_ds = f.create_dataset('data', shape=(count, size_input, size_input, 1), maxshape=(None, size_input, size_input, 1),
                               dtype='f4', chunks=True)
    label_ds = f.create_dataset('label', shape=(count, size_label, size_label, 1), maxshape=(None, size_label, size_label, 1),
                                dtype='f4', chunks=True)

    # Writing data in chunks
    for batchno in range(0, count, chunksz):
        batchdata = data[batchno:batchno+chunksz]
        batchlabs = label[batchno:batchno+chunksz]

        startloc = totalct
        endloc = totalct + batchdata.shape[0]

        data_ds[startloc:endloc] = batchdata[..., np.newaxis]
        label_ds[startloc:endloc] = batchlabs[..., np.newaxis]

        totalct = endloc

    print(f"Total count of data stored: {totalct}")

# Function to crop images to be divisible by modulo
def modcrop(imgs, modulo):
    if len(imgs.shape) == 2:  # Grayscale
        sz = np.array(imgs.shape)
        sz = sz - (sz % modulo)
        imgs = imgs[:sz[0], :sz[1]]
    else:  # RGB or other
        sz = np.array(imgs.shape[:2])
        sz = sz - (sz % modulo)
        imgs = imgs[:sz[0], :sz[1], :]
    return imgs


Total count of data stored: 21884


Model

In [ ]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Input
from tensorflow.keras.optimizers import Adam

# Define SRCNN model
def build_srcnn_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(64, (9, 9), kernel_initializer="he_normal", padding="valid"))
    model.add(Activation("relu"))
    model.add(Conv2D(32, (1, 1), kernel_initializer="he_normal", padding="valid"))
    model.add(Activation("relu"))
    model.add(Conv2D(1, (5, 5), kernel_initializer="he_normal", padding="valid"))
    return model


# Load HDF5 data and reshape to "channels last" format
def load_h5_data(h5_file_path, data_key='data', label_key='label'):
    with h5py.File(h5_file_path, 'r') as f:
        data = np.array(f[data_key])
        labels = np.array(f[label_key])
    # Reshape data to (batch_size, height, width, channels)
    data = data.reshape(-1, 33, 33, 1)
    labels = labels.reshape(-1, 21, 21, 1)
    return data, labels


# Path to training .h5 file
h5_file_path_train = '/content/Results/train.h5'

# Load training data
train_images, train_labels = load_h5_data(h5_file_path_train)

# Check the shape of your data
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)

# Set input shape and build the model
input_shape = (33, 33, 1)
model = build_srcnn_model(input_shape)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

# Convert training data to tf.data.Dataset
def create_tf_dataset(images, labels, batch_size=128):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

# Create training dataset
train_dataset = create_tf_dataset(train_images, train_labels, batch_size=128)

# Train the model using only training data
model.fit(train_dataset, epochs=10, verbose=1)


Train images shape: (21884, 33, 33, 1)
Train labels shape: (21884, 21, 21, 1)
Epoch 1/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.1919
Epoch 2/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0043
Epoch 3/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0025
Epoch 4/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0022
Epoch 5/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0020
Epoch 6/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0019
Epoch 7/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0018
Epoch 8/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0018
Epoch 9/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0018
Epoch 10/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0017


In [ ]:
# initialize the batch size and number of epochs for training
BATCH_SIZE = 128
NUM_EPOCHS = 10
# initialize the scale (the factor in which we want to learn how to
# enlarge images by) along with the input width and height dimensions
# to our SRCNN
SCALE = 3.0
INPUT_DIM = 33
# the label size should be the output spatial dimensions of the SRCNN
# while our padding ensures we properly crop the label ROI
LABEL_SIZE = 21
PAD = int((INPUT_DIM - LABEL_SIZE) / 2.0)
# the stride controls the step size of our sliding window
STRIDE = 21

In [ ]:
import cv2
import numpy as np
import h5py
import PIL
import os
from PIL import Image

In [ ]:
model.save('model1.h5')

In [ ]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Input
from tensorflow.keras.optimizers import Adam
loaded_model = tf.keras.models.load_model('/content/model1.h5')

In [ ]:
loaded_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 25, 25, 64)          │           5,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 25, 25, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 25, 25, 32)          │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 25, 25, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 21, 21, 1)           │             801 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,131 (31.77 KB)

 Trainable params: 8,129 (31.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
input_shape = loaded_model.input_shape
output_shape = loaded_model.output_shape

print(f"Input shape: {input_shape}")
print(f"Output shape: {output_shape}")

Input shape: (None, 33, 33, 1)
Output shape: (None, 21, 21, 1)


In [ ]:
#applying model to images in set14
print("[INFO] generating image...")
image = cv2.imread('/content/drive/MyDrive/Data/Test/butterfly_GT.bmp')
image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
image=image[:, :, 0]
(h, w) = image.shape[:2]
w -= int(w % SCALE)
h -= int(h % SCALE)
#image = image[0:h, 0:w]
# resize the input image using bicubic interpolation then write the
# baseline image to disk
lowW = int(w * (1.0 / SCALE))
lowH = int(h * (1.0 / SCALE))
highW = int(lowW * (SCALE / 1.0))
highH = int(lowH * (SCALE / 1.0))
scaled = np.array(Image.fromarray(image).resize((lowW, lowH),
    resample=PIL.Image.BICUBIC))
scaled = np.array(Image.fromarray(scaled).resize((highW, highH),
    resample=PIL.Image.BICUBIC))
cv2.imwrite('/content/Results/sclared.bmp', scaled)
# allocate memory for the output image
output = np.zeros(scaled.shape)
(h, w) = output.shape[:2]
# slide a window from left-to-right and top-to-bottom
for y in range(0, h - INPUT_DIM + 1, LABEL_SIZE):
    for x in range(0, w - INPUT_DIM + 1, LABEL_SIZE):
        # crop ROI from our scaled image
        crop = scaled[y:y + INPUT_DIM,
            x:x + INPUT_DIM].astype("float32")
        # make a prediction on the crop and store it in our output
        # image
        P = loaded_model.predict(np.expand_dims(crop, axis=0))
        P = P.reshape((LABEL_SIZE, LABEL_SIZE, 1))
        P = P.squeeze()
        output[y + PAD:y + PAD + LABEL_SIZE,x + PAD:x + PAD + LABEL_SIZE] = P
# remove any of the black borders in the output image caused by the
# padding, then clip any values that fall outside the range [0, 255]
#output = output[PAD:h - ((h % INPUT_DIM) + PAD),PAD:w - ((w % INPUT_DIM) + PAD)]
output = np.clip(output, 0, 255).astype("uint8")
# write the output image to disk
cv2.imwrite('/content/Results/result2.bmp', output)

[INFO] generating image...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━

True

[INFO] generating image...


True

In [ ]:
## edited code
import cv2
import numpy as np
from PIL import Image

# Parameters
SCALE = 2  # Upscaling factor
INPUT_DIM = 33  # Input patch size for model
LABEL_SIZE = 21  # Output patch size from model
PAD = (INPUT_DIM - LABEL_SIZE) // 2  # Padding around the crop

# Load and preprocess the image
print("[INFO] generating image...")
image = cv2.imread('/content/drive/MyDrive/Data/Test/butterfly_GT.bmp')
image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
image = image[:, :, 0]  # Use the luminance (Y) channel

(h, w) = image.shape[:2]

# Calculate 25% padding for height and width
pad_h = int(h * 0.25)
pad_w = int(w * 0.25)

# Pad the image by 0.25 of its original size on all sides
padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')

#padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
padded_h, padded_w = padded_image.shape

# Resize to simulate low-resolution and then bicubic upscale for comparison
lowW = int(w * (1.0 / SCALE))
lowH = int(h * (1.0 / SCALE))
scaled = np.array(Image.fromarray(image).resize((lowW, lowH), resample=Image.BICUBIC))
scaled = np.array(Image.fromarray(scaled).resize((w, h), resample=Image.BICUBIC))
cv2.imwrite('/content/Results/scaled.bmp', scaled)

# Allocate memory for the output image based on padded dimensions
output = np.zeros((padded_h, padded_w), dtype="float32")
count_matrix = np.zeros((padded_h, padded_w), dtype="float32")  # to handle overlaps

# Slide a window across the padded image
for y in range(0, padded_h - INPUT_DIM + 1, LABEL_SIZE):
    for x in range(0, padded_w - INPUT_DIM + 1, LABEL_SIZE):
        # Extract a patch from the padded image
        crop = padded_image[y:y + INPUT_DIM, x:x + INPUT_DIM].astype("float32")

        # Make a prediction on the crop and reshape the output
        P = loaded_model.predict(np.expand_dims(crop, axis=0))
        P = P.reshape((LABEL_SIZE, LABEL_SIZE))  # Adjust based on model output

        # Place the predicted patch into the output image
        output[y + PAD:y + PAD + LABEL_SIZE, x + PAD:x + PAD + LABEL_SIZE] += P
        count_matrix[y + PAD:y + PAD + LABEL_SIZE, x + PAD:x + PAD + LABEL_SIZE] += 1

# Normalize by count_matrix to handle overlapping areas
output = np.divide(output, count_matrix, out=np.zeros_like(output), where=count_matrix!=0)

# Remove padding, clip values, and save the final output
output = np.clip(output[pad_h:-pad_h, pad_w:-pad_w], 0, 255).astype("uint8")
cv2.imwrite('/content/Results/result.bmp', output)


[INFO] generating image...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━

True